 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8887/notebooks/rnn_test.ipynb#Set-up-data" data-toc-modified-id="Set-up-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set up data</a></span></li><li><span><a href="http://localhost:8887/notebooks/rnn_test.ipynb#Build-model" data-toc-modified-id="Build-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Build model</a></span></li></ul></div>

In [99]:
# Imports
from importlib import reload
import crps_loss; reload(crps_loss)
from crps_loss import crps_cost_function, crps_cost_function_seq
import utils; reload(utils)
from utils import *
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.layers import Input, Dense, merge, Embedding, Flatten, Dropout, \
    SimpleRNN, LSTM, TimeDistributed, GRU, Dropout
from keras.layers.merge import Concatenate
from keras.models import Model, Sequential
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

In [2]:
keras.backend.backend()

'tensorflow'

In [3]:
# Basic setup
# DATA_DIR = '/Volumes/STICK/data/ppnn_data/'  # Mac
DATA_DIR = '/project/meteo/w2w/C7/ppnn_data/'   # LMU
results_dir = '../results/'
window_size = 25   # Days in rolling window
fclt = 48   # Forecast lead time in hours

## Set up data

In [4]:

train_dates = ['2015-01-01', '2016-01-01']
test_dates =  ['2016-01-01', '2017-01-01']
train_set, test_set = get_train_test_sets(DATA_DIR, train_dates, test_dates)

train set contains 365 days
test set contains 366 days


In [5]:
train_set.features.shape

(180849, 2)

In [6]:
targets, features, dates, station_id, feature_names = load_raw_data(DATA_DIR)

In [7]:
targets.shape, features.shape

((3653, 537), (2, 3653, 537))

In [8]:
train_dates_idxs = [return_date_idx(dates, train_dates[0]),
                            return_date_idx(dates, train_dates[1])]
test_dates_idxs = [return_date_idx(dates, test_dates[0]),
                           return_date_idx(dates, test_dates[1])]

In [9]:
train_dates_idxs

[2920, 3285]

In [10]:
x_train = features[:, train_dates_idxs[0]:train_dates_idxs[1]]
y_train = targets[train_dates_idxs[0]:train_dates_idxs[1]]

In [11]:
x_train.shape, y_train.shape

((2, 365, 537), (365, 537))

In [12]:
x_test = features[:, test_dates_idxs[0]:test_dates_idxs[1]]
y_test = targets[test_dates_idxs[0]:test_dates_idxs[1]]

In [13]:
x_test.shape, y_test.shape

((2, 366, 537), (366, 537))

In [63]:
def prep_seq_data(x, y, seq_len):
    # Stack sequences [seq, feature, day, station]
    x_seq = np.stack([x[:, i:-(seq_len-i)] for i in range(seq_len)])
    y_seq = np.stack([y[i:-(seq_len-i)] for i in range(seq_len)])
    
    # Ravel arrays [seq, feature, instance]
    x_seq = np.reshape(x_seq, (seq_len, 2, -1))
    y_seq = np.reshape(y_seq, (seq_len, -1))
    
    # Roll arrays[sample, time step, feature]
    x_seq = np.rollaxis(x_seq, 2, 0)
    y_seq = np.rollaxis(y_seq, 1, 0)
    y_seq = np.atleast_3d(y_seq)
    
    # Remove nans
    mask = np.isfinite(np.sum(y_seq, axis=(1,2)))
    x_seq = x_seq[mask]
    y_seq = y_seq[mask]
    
    return x_seq, y_seq.data

In [75]:
# Reshape them
seq_len = 5

In [76]:
x_seq_train, y_seq_train = prep_seq_data(x_train, y_train, seq_len)

In [77]:
x_seq_test, y_seq_test = prep_seq_data(x_test, y_test, seq_len)

In [78]:
x_seq_train.shape, y_seq_train.shape

((177468, 5, 2), (177468, 5, 1))

In [79]:
x_seq_train[0]

array([[ 3.03763652,  0.80634564],
       [ 3.66275382,  0.67297047],
       [ 4.54295206,  0.37079421],
       [ 2.89147758,  0.83139253],
       [ 1.78039348,  0.75268936]], dtype=float32)

In [80]:
y_seq_train[0]

array([[ 2.20000005],
       [ 4.69999981],
       [ 5.0999999 ],
       [ 2.9000001 ],
       [ 1.39999998]], dtype=float32)

## Build model

In [102]:
batch_size = 64
hidden_nodes = 100

In [103]:
inp = Input(shape=(seq_len, 2, )) # time step, feature
x = GRU(hidden_nodes, return_sequences=True)(inp)
x = Dropout(0.5)(x)
x = TimeDistributed(Dense(2, activation='linear'))(x)
rnn_model = Model(inputs=inp, outputs=x)

In [104]:
rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 5, 2)              0         
_________________________________________________________________
gru_9 (GRU)                  (None, 5, 100)            30900     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 100)            0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 5, 2)              202       
Total params: 31,102
Trainable params: 31,102
Non-trainable params: 0
_________________________________________________________________


In [105]:
rnn_model.compile(optimizer=Adam(0.01), loss=crps_cost_function_seq)

In [106]:
rnn_model.fit(x_seq_train, y_seq_train, epochs=10, batch_size=1024,
              validation_data=(x_seq_test, y_seq_test))
#rnn_model.fit(x_seq_train, y_seq_train, epochs=10, batch_size=1024)

Train on 177468 samples, validate on 178975 samples
Epoch 1/10
177468/177468 [==============================] - 5s - loss: 1.5279 - val_loss: 1.0545
Epoch 2/10
177468/177468 [==============================] - 4s - loss: 1.2094 - val_loss: 1.0369
Epoch 3/10
177468/177468 [==============================] - 4s - loss: 1.1881 - val_loss: 1.0421
Epoch 4/10
177468/177468 [==============================] - 4s - loss: 1.1714 - val_loss: 1.0342
Epoch 5/10
177468/177468 [==============================] - 4s - loss: 1.1593 - val_loss: 1.0338
Epoch 6/10
177468/177468 [==============================] - 4s - loss: 1.1472 - val_loss: 1.0310
Epoch 7/10
177468/177468 [==============================] - 4s - loss: 1.1403 - val_loss: 1.0316
Epoch 8/10
177468/177468 [==============================] - 4s - loss: 1.1338 - val_loss: 1.0344
Epoch 9/10
177468/177468 [==============================] - 4s - loss: 1.1272 - val_loss: 1.0458
Epoch 10/10
177468/177468 [==============================] - 4s - loss: 1.1

In [92]:
inp = Input(shape=(seq_len, 2, )) # time step, feature
x = GRU(hidden_nodes)(inp)
x = Dense(2, activation='linear')(x)
rnn_model2 = Model(inputs=inp, outputs=x)

In [93]:
rnn_model2.compile(optimizer=Adam(0.001), loss=crps_cost_function)

In [94]:
rnn_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 5, 2)              0         
_________________________________________________________________
gru_7 (GRU)                  (None, 20)                1380      
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 42        
Total params: 1,422
Trainable params: 1,422
Non-trainable params: 0
_________________________________________________________________


In [98]:
rnn_model2.fit(x_seq_train, y_seq_train[:,-1], epochs=5, batch_size=1024,
              validation_data=(x_seq_test, y_seq_test[:,-1]))
#rnn_model2.fit(x_seq_train, y_seq_train[:,-1], epochs=10, batch_size=1024)

Train on 177468 samples, validate on 178975 samples
Epoch 1/5
177468/177468 [==============================] - 3s - loss: 1.0515 - val_loss: 1.0134
Epoch 2/5
177468/177468 [==============================] - 3s - loss: 1.0436 - val_loss: 1.0097
Epoch 3/5
177468/177468 [==============================] - 3s - loss: 1.0383 - val_loss: 1.0100
Epoch 4/5
177468/177468 [==============================] - 3s - loss: 1.0343 - val_loss: 1.0104
Epoch 5/5
177468/177468 [==============================] - 3s - loss: 1.0314 - val_loss: 1.0101
